https://www.kaggle.com/code/imthebaron/face-training?scriptVersionId=198879052

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob as gb
import pickle
import cv2
import pandas as pd
from tqdm import tqdm

In [ ]:
data_path = '/kaggle/input/cvpr-faces/augmented_faces'

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64

In [ ]:
classes = []
class_count = []
train_examples = 0

for f in tqdm(os.listdir(data_path)):
  files = gb.glob(pathname = str(data_path + '//' + f + '//' + '/*'))
  classes.append(f)
  class_count.append(len(files))
  train_examples += len(files)

sns.barplot(x=classes, y=class_count)
print(train_examples)

In [ ]:
train_gen = ImageDataGenerator(
      #rescale=1.0/255,
      #preprocessing_function = tf.keras.applications.vgg16.preprocess_input
      #preprocessing_function = tf.keras.applications.mobilenet.preprocess_input,
      preprocessing_function = tf.keras.applications.densenet.preprocess_input,
      validation_split = 0.3
)

In [ ]:
train_batches = train_gen.flow_from_directory(
          directory = data_path,
          target_size = (IMG_SIZE, IMG_SIZE),
          class_mode='sparse',
          batch_size = BATCH_SIZE,
          shuffle=True,
          seed=42,
          subset = 'training'
      )

valid_batches = train_gen.flow_from_directory(
    directory = data_path,
    target_size = (IMG_SIZE, IMG_SIZE),
    class_mode='sparse',
    shuffle=True,
    seed=42,
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

In [ ]:
class_indices = train_batches.class_indices  # Dictionary mapping class names to integer labels

# Step 2: Invert the class_indices dictionary to get a mapping from labels to class names
class_names = {v: k for k, v in class_indices.items()}

# Output class names
print(class_names)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

base_model = tf.keras.applications.DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

def custom():  # Change to 3 channels
    # Load Xception model with pre-trained weights (on ImageNet)
    input = layers.Input(shape=(224, 224, 3))

    x = base_model(input, training=True)

    # Add custom layers on top of the base model
    x = layers.GlobalAveragePooling2D()(x)  # Change to Global Average Pooling
    x = layers.Dropout(0.3)(x)  # Regularization
    x = layers.Dense(1024, activation='relu')(x)  # Dense layer  # Regularization
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    output = x = layers.Dense(56, activation='softmax')(x)

    # Create the model
    model = tf.keras.Model(inputs=input, outputs=output, name='bengali.ai')
    return model

# Create and compile the model
model = custom()
model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # Metric to monitor (can be 'loss', 'accuracy', etc.)
    factor=0.9,           # Factor by which the learning rate will be reduced
    patience=20,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6           # Lower bound on the learning rate
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation accuracy
    min_delta=0.001,         # Minimum change in monitored value to qualify as improvement
    patience=4,             # Stop after 10 epochs of no improvement
    mode='min',              # Maximize the validation accuracy
    restore_best_weights=True,  # Restore model weights from the best epoch
    verbose=1
)

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoint.weights.h5',
    monitor='val_loss',
    mode='min',
    save_freq = 'epoch',
    save_weights_only=True,
    save_best_only=True)

In [ ]:
train_history = model.fit(
    train_batches,
    epochs = 50,
    steps_per_epoch = 20055 // BATCH_SIZE,
    validation_data = valid_batches,
    validation_steps = 8553 // BATCH_SIZE,
    callbacks=[model_checkpoint_callback, early_stopping_callback]#, reduce_lr]
)

In [ ]:
model.save('face densenet121.h5')

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize = (10, 5))
plt.plot(train_history.history['loss'], '--o', label='train loss')
plt.plot(train_history.history['val_loss'], '--o', label='val loss')
plt.legend()
plt.title('training loss & val loss')
plt.savefig('fig_total_loss.png', format='png', dpi=400)
plt.show()

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize = (10, 5))
plt.plot(train_history.history['accuracy'], '--o', label='train accuracy')
plt.plot(train_history.history['val_accuracy'], '--o', label='val accuracy')
plt.legend()
plt.title('training accuracy & val accuracy')
plt.savefig('fig_total_acc.png', format='png', dpi=400)
plt.show()

In [ ]:
pd.DataFrame(train_history.history).to_csv('history.csv', index=False)

In [ ]:
df = pd.read_csv('/kaggle/working/history.csv')
print(df.shape)
df.head(df.shape[0])

In [ ]:
# model = tf.keras.models.load_model('/kaggle/input/face-training/custom.h5')
# model.summary()

In [ ]:
import numpy as np

img = tf.keras.utils.load_img('/kaggle/input/cvpr-faces/faces/Rono/4.png', target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img)
img = tf.keras.applications.densenet.preprocess_input(img_array)
img_array = tf.expand_dims(img_array, 0)

img_predictions = model.predict(img_array)
print([np.argmax(np.round(img_predictions,2))])
pred_label = class_names[np.argmax(np.round(img_predictions,2))]
print(" Predicted label is:"+ pred_label)

plt.imshow(img)